<a href="https://colab.research.google.com/github/jimhaines37/DataScience/blob/main/_demos/Demo-04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Demo 04 - Pandas, Data Tables, and Tidy Data

In this demo we will go over some of the basics of Pandas, so that you have examples of the basic functionality for reference. We then move into working with those data tables, a deep dive on the GroupBy command and how it works, and finish with another deep dive on Melting data and the concepts of Tidy Data.

In [ ]:
# first, mount your google drive, change to the course folder, pull latest changes, and change to the lab folder.
# Startup Magic to: (1) Mount Google Drive
# (2) Change to Course Folder
# (3) Pull latest Changes
# (4) Move to the Demo Directory so that the data files are available

from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/My Drive/cmps3160
!git pull
%cd _demos

In [ ]:
# Includes and Standard Magic...
### Standard Magic and startup initializers.

# Load Numpy
import numpy as np
# Load MatPlotLib
import matplotlib
import matplotlib.pyplot as plt
# Load Pandas
import pandas as pd

# This lets us show plots inline and also save PDF plots if we want them
%matplotlib inline
from matplotlib.backends.backend_pdf import PdfPages
matplotlib.style.use('fivethirtyeight')

# These two things are for Pandas, it widens the notebook and lets us display data easily.
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

# Show a ludicrus number of rows and columns
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

## Basic Pandas Functionality

### Creating Tables

First, let's take a look at some basic Pandas functionality. These are small examples to show the ideas of creating, selecting from, and working with data tables.

In [ ]:
s1 = pd.DataFrame([0.3, 0.8, 0.1, 4.0, 1.2], 
                  index = ['a', 'c', 'd', 'e', 'f'], 
                  columns=['data'])

display(s1)

In [ ]:
s2 = pd.DataFrame([0.9, 0.1, 4.8, 0.3], 
                  index = ['b', 'c', 'd', 'g'], 
                  columns=['data'])
display(s2)

Re-index can get used to do some funky things or make bigger frames.. this is not common usage but you use reindex to make new entries in an already existing table.  [Doc Page](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.reindex.html)

```Places NA/NaN in locations having no value in the previous index. A new object is produced unless the new index is equivalent to the current one and copy=False.```

In [ ]:
# We can use re-index to fill in some gaps if we want..
import string

display(s1.reindex(list(string.ascii_lowercase)[:10]))

# We can also use the 'ffill' method to fill the missing data with the closest data.
display(s1.reindex(list(string.ascii_lowercase)[:10], method='ffill'))

Note that the above operation did not happen in place! so that our s1 table is preserved!

In [ ]:
display(s1)

### Filtering, Selecting, and Setting Data.

The first question we might have is how do we get at data in our dataframe? The easiest way is to get a location by it's `index` or `label`. To do this we use the `.loc` command. We can use this command to slice by the index as well!

For more on the difference between loc and iloc see the [10 Mins to Pandas](https://pandas.pydata.org/docs/user_guide/10min.html) documentation.

In [ ]:
# Selection by location or loc

s1.loc['f']

In [ ]:
s1.loc['a':'d']

In [ ]:
s2.loc['b']

We can also use the `iloc` command to get a row by it's **interger position**. 

In [ ]:
# Selection by position, iloc
s2.iloc[0]

In [ ]:
display(s1.iloc[0:3])

**Gotcha Warning!** Note that loc is inclusive while iloc is not inclusive at the end of the range -- just like indexing into an array!

In [ ]:
display(s1)
display(s1.iloc[1:4])


We can use set data in a data frame by using the `at` and `iat` commands.

In [ ]:
display(s1)
s1.at['a','data'] = 0
display(s1)

In [ ]:
display(s1)
s1.iat[2,0] = 0
display(s1)

Note that both of the above operations happened inplace!! **Why?**

In [ ]:
# Let's reset our data frames since we messed them up!

s1 = pd.DataFrame([0.3, 0.8, 0.1, 4.0, 1.2], 
                  index = ['a', 'c', 'd', 'e', 'f'], 
                  columns=['data'])

s2 = pd.DataFrame([0.9, 0.1, 4.8, 0.3], 
                  index = ['b', 'c', 'd', 'g'], 
                  columns=['data'])

In [ ]:
# Recall we can also use aggregation operators over the columns

s2['data'].sum()

As we have seen we can use boolean operations to filter and select data. Recall that these operations return a **view** of the data and not a copy of the dataframe.

In [ ]:
s2[s2['data'] > 3]

In [ ]:
s1[ (s1['data'] < 1.0) & (s1['data'] > 3.0) ]

In [ ]:
s1

### Operations on Data Tables - Broadcasting

When we add constants to a table we get something called **broadcasting** where the operation happens to every element of a table (or column!) You can read more about this essential basic functionality at the [Pandas Documentation Website](https://pandas.pydata.org/docs/user_guide/basics.html)

In [ ]:
s1 + 0.5

In [ ]:
s2

Note that the above operations did not happen **inplace** so that our tables remain the same as when we declared them! When you manipulate a datatable you must always tell pandas to either save that as a new variable or table, or to do the operation in place.

**Question:** What happens if we add two tables together?

In [ ]:
# Note again that these operations did not happen in place!
display(s1)
display(s2)

In [ ]:
s1 + s2

In [ ]:
s1 - s2

Note that + is like what we'll learn is an inner join soon but it also adds the numbers together!

You can also apply functions across and down and with lambdas...

In [ ]:
frame = pd.DataFrame(np.random.randn(4,3), columns=list('abc'),
                 index=['Utah', 'Ohio', 'Texas','Oregon'])
display(frame)

In [ ]:
frame.abs()

In [ ]:
minmax = lambda x: x.max() - x.min()

frame.apply(minmax)

In [ ]:
# Default is axis=0 or per column, can also do per row!
frame.apply(minmax, axis=1)


### Operations on Data Tables - Ranking

Some of the [Pandas Rank](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.rank.html) commands aren't totally obvious...

In [ ]:
frame = pd.DataFrame({'a':[0, 1, 0, 1],
                      'b':[4.3, 7, -3, 2],
                   'c':[-2, 5, 8, -2.5]})
display(frame)

In [ ]:
# Default is average but you can do other things!

display(frame.rank(ascending=False))

display(frame.rank(ascending=False, method='first'))

In [ ]:
# Can also sort along rows!

frame.rank(axis=1)

### Using Hierarchical Indices

To set a **hierarchical index** one can refer to the [documetnation page](https://pandas.pydata.org/pandas-docs/stable/user_guide/advanced.html).

In [ ]:
# Setting a heiararichal index -- The bad way would be to build up tuples as an index.

index = [('California', 2000), ('California', 2010),
         ('New York', 2000), ('New York', 2010),
         ('Texas', 2000), ('Texas', 2010)]
populations = [33871648, 37253956,
               18976457, 19378102,
               20851820, 25145561]
pop = pd.Series(populations, index=index)

In [ ]:
display(pop)

In [ ]:
# Now we can still get slices etc. in a strightforward way.
pop[('California', 2010):('Texas', 2000)]

In [ ]:
# But if we want to get all 2010 data we have to do something like...
pop[[i for i in pop.index if i[1] == 2010]]

In [ ]:
#We can make the data frame as (note it is tidy!)
df = pd.DataFrame([('California', 2000, 33871648),
                   ('California', 2010, 37253956),
                   ('New York', 2000, 18976457),
                   ('New York', 2010, 19378102),
                   ('Texas', 2000, 20851820),
                   ('Texas', 2010, 25145561)],
                  columns=['state', 'year', 'pop'])
df

In [ ]:
df.set_index(['state', 'year'], inplace=True)
df

In [ ]:
# And now we can do cool stuff slicing, but it gets compicated with tuples.
df.loc[('California')]

In [ ]:
df.index

The [df.xs](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.xs.html) command can help with slicing multi-indicies.

In [ ]:
# Thought this can get a bit complicated... 
df.xs(2010, level=1)

### Importance of using np.nan

Note that we have to import numpy.nan but once we do we can use the groupby and other methods without having to worry about what to do with missing data.

For the example below, what happens when we don't use NaNs in the data table?

In [ ]:
# Make a data frame from a lists
# Try replacing 'XX' with various values..

df = pd.DataFrame({'age':     [12.2, 11.0, 15.6, '--'],
                  'wgt_kg':   [42.3, 40.8, 65.3, 84.2],
                  'hgt_cm':   [145.1, 143.8, 165.3, 185.8],
                  'sex':      ['male', 'female', 'male', 'male'],
                  'country': ['USA', 'AUS', 'EU', 'USA']})
df

In [ ]:
df['age'].describe()

In [ ]:
# Note the type that has been imputed here...
df.dtypes

In [ ]:
df['age'].sum()

In [ ]:
# Need to make sure it's set as number!

df["age"] = pd.to_numeric(df["age"])

In [ ]:
# But first we have to make sure it's a NAN!!
display(df["age"].replace("--", np.nan))

df["age"].replace("--", np.nan,inplace=True)

In [ ]:
df.dtypes

In [ ]:
df["age"].sum()

## The Groupby Command

Below we see what happens in Pandas when we use the groupby command.

In [ ]:
df = pd.DataFrame({'age':     [12.2, 11.0, 15.6, 35.1],
                  'wgt_kg':   [42.3, 40.8, 65.3, 84.2],
                  'hgt_cm':   [145.1, 143.8, 165.3, 185.8],
                  'sex':      ['male', 'female', 'male', 'male'],
                  'country': ['USA', 'AUS', 'EU', 'USA']})
df

In [ ]:
# This is a little weird, recall that we have lazy evaluation!

df.groupby(['sex'])

In [ ]:
# If we describe it then we can force python to not be lazy!

df.groupby('sex').describe()

In [ ]:
# We can pass operators to groupby to get better results.
df.groupby(['sex']).mean()

In [ ]:
# Can also group by multiple columns
df.groupby(['sex', 'country']).describe()


In [ ]:
# Multi-Index!!
# We can select based on a list of indices
df.groupby(['sex', 'country']).mean().loc['female', 'AUS']

In [ ]:
## We can also limit ourselves by just subselecting columns
display(df.groupby(["country"]).count())
display(df.groupby(["country"])[["sex"]].count())

In [ ]:
# same as...
df['country'].value_counts()

In [ ]:
# This is the same as...
grouped = df.groupby(['country'])

In [ ]:
type(grouped)

To get access to a group that we made, use the [get_group() command](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.core.groupby.GroupBy.get_group.html).

We can also get access to the actual tuples and they're types that are generated by the `groupby` commands

In [ ]:
grouped.groups

In [ ]:
grouped.mean()

In [ ]:
grouped.get_group('AUS')

In [ ]:
# Or do it fancy?
[display(grouped.get_group(i)) for i in grouped.groups]

### A Big Deep Dive on Groupby!

**Question** If we were going to implement Groupby, how would you do it?

The first thing we can note is that if we iterate over the groups, it makes it clear what Pandas and Python are doing.

However we don't want to iterate ourselves as this will generally be slow and error prone!

**We always want to replace for loops in Pandas** since Pandas functions are highly optimized (numpy + C + vectorization)

In [ ]:
grouped = df.groupby(['country'])
grouped.groups

In [ ]:
for group_index, groupi in grouped:
  print(group_index)
  display(groupi)
  print(groupi['age'].mean())

In [ ]:
df.groupby('country')['age'].mean()

### Split-Apply-Combine

The `groupby` operation can be understood as a Split-Apply-Combine pattern:

<img src="https://jakevdp.github.io/figures/split-apply-combine.svg" width=600/>

How could we implement this?

In [ ]:
dfi = pd.DataFrame([
    {'key': 'A', 'data': 1}, {'key': 'B', 'data': 2},
    {'key': 'C', 'data': 3}, {'key': 'A', 'data': 4},
    {'key': 'B', 'data': 5}, {'key': 'C', 'data': 6},
  ]
)
dfi

In [ ]:
dfi.groupby('key').sum()

In [ ]:
# Option 1: Boolean Masks
def group_by_masks(dfi):
  result = []
  for key in ['A','B', 'C']:
    result.append({
        'key': key,
        'data': dfi[dfi['key']==key]['data'].sum()
    })
  return pd.DataFrame(result)

group_by_masks(dfi)

What is the Big Oh runtime of this implementation?

Can we improve?

In [ ]:
# Option 2: Build a dict from key->sum in a single pass through data.
from collections import defaultdict
def group_by_dict(dfi):
  sums = {}
  for index, row in dfi.iterrows():
    sums[row['key']] = sums.get(row['key'], 0) + row['data']
  # display(sums)
  return pd.DataFrame.from_dict(sums, orient='index', columns=['data'])
  return sums
group_by_dict(dfi)

### Lazy evaluation

- `groupby` doesn't actually do much until you do something with the groups.

In [ ]:
# make a big DataFrame.
bigdfi = dfi.sample(100000, replace=True)

In [ ]:
bigdfi

In [ ]:
# call groupby, but don't do anything with it.
%timeit bigdfi.groupby('key')

In [ ]:
# once you use the groups, it takes a lot more time.
%timeit bigdfi.groupby('key').sum()

In [ ]:
# for comparison, this is the time to just sum the data column.
%timeit bigdfi.data.sum()

In [ ]:
# Pandas uses Cython internally, which implements core python functions 
# # in highly optimized C code.
# This is one reason why our implementation is horribly slow.
%timeit group_by_dict(bigdfi)

In [ ]:
# our mask implementation relies on pandas functions, so it is fast,
# even though the asymptotic runtime should be larger than group_by_dict!
%timeit group_by_masks(bigdfi)

In [ ]:
# Pandas builds on numpy 
# See: "Why is numpy fast?" https://numpy.org/doc/stable/user/whatisnumpy.html
import numpy as np
a = np.random.rand(100000)
b = np.random.rand(100000)

In [ ]:
def dot_product_slow(a,b):
  dot = 0
  for i in range(len(a)):
    dot += a[i] * b[i]
  return dot

%timeit dot_product_slow(a,b)

In [ ]:
%timeit np.dot(a,b)

Futher reading on optimization:
- [Locality of reference](https://en.wikipedia.org/wiki/Locality_of_reference)
- [Vectorization](https://en.wikipedia.org/wiki/Automatic_vectorization)
- [Broadcasting](https://numpy.org/doc/stable/user/basics.broadcasting.html)

## Melting and Tidy Data

In this part of the demo we will duplicate the slides and get some practice with the Melt command to make a data table tidy where it wasn't before!

In [ ]:
df = pd.read_csv('./data/religon.csv')
df

The [melt command](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.melt.html) does...

```
DataFrame.melt(id_vars=None, value_vars=None, var_name=None, value_name='value', col_level=None, ignore_index=True)

Unpivot a DataFrame from wide to long format, optionally leaving identifiers set.

This function is useful to massage a DataFrame into a format where one or more columns are identifier variables (id_vars), while all other columns, considered measured variables (value_vars), are “unpivoted” to the row axis, leaving just two non-identifier columns, ‘variable’ and ‘value’.
```

So here we want to take the above data frame and make it `grow down` by converting income into a categorical variable.

In [ ]:
f_df = pd.melt(df,
               ["religion"],
               var_name="income",
               value_name="freq")
f_df = f_df.sort_values(by=["income"])
f_df.head(10)

In [ ]:
# We can now use more simple slicing to see the things we need.

f_df[(f_df['religion'] == 'Atheist')].sort_values(by='income')

In [ ]:
f_df[(f_df['income'] == '$50-75k')].sort_values(by='freq', ascending=False)

### An Over The Top Example

In this example we really walk through all the nitty gritty of taking the Billboard data and melting it down, fixing some errors in the table, and getting a dataframe that we can do easier manipulation on.

In [ ]:
b_df = pd.read_csv("./data/billboard.csv")
b_df.head()

This thing is a mess!! What if want to say, graph the positions of a song for a given artist? Then how would we do it?

Again, we are going to melt to make the table `grow down` by keeping the `id_vars` and then melting the rows based on week position to be values.



In [ ]:
# Keep identifier variables
id_vars = ["year",
           "artist.inverted",
           "track",
           "time",
           "genre",
           "date.entered",
           "date.peaked"]

# Melt the rest into week and rank columns
b_df = pd.melt(frame=b_df,
             id_vars=id_vars,
             var_name="week",
             value_name="rank")


In [ ]:
display(b_df.head(20))
b_df.dtypes

In [ ]:
# Let's fix the week thing, it's not good...
b_df["week"] = b_df['week'].str.extract('(\d+)', expand=False).astype(int)

In [ ]:
b_df[:5]

In [ ]:
b_df["week"].unique()

There's a subtle problem here, rank is an int but some are NAN's and we can't have [int-nan's...](https://pandas.pydata.org/pandas-docs/version/0.24/whatsnew/v0.24.0.html#optional-integer-na-support)

In [ ]:
# Why not ints? Gotcha! ?!?!?
b_df["rank"] = b_df["rank"].astype(float)

display(b_df.head(20))

In [ ]:
# We can see if anything is missing...
# Remember axis=1 goes row wise..so we are saying show us any row with missing values...
b_df[b_df.isnull().any(axis=1)]

In [ ]:
# We don't need these so... let's drop them.
b_df = b_df.dropna()

We're going to use the handy [to_datetime](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.to_datetime.html) which will get us a date!

In [ ]:
# Create "date" columns
#  date = (date entered chart) + (# of weeks) - (1 week) [fence post problem]
b_df['date'] = pd.to_datetime(b_df['date.entered']) + pd.to_timedelta(b_df['week'], unit='w') - pd.DateOffset(weeks=1) 

In [ ]:
display(b_df[100:110])
b_df.dtypes

In [ ]:
# Ignore now-redundant, messy columns -- same as dropping 
b_df = b_df[["year",
         "artist.inverted",
         "track",
         "time",
         "genre",
         "week",
         "rank",
         "date"]]

b_df = b_df.sort_values(ascending=True, by=["year","artist.inverted","track","week","rank"])

# Keep tidy dataset for future usage
billboard = b_df

billboard.head(10)

In [ ]:
# Now we can see the artists we have...
billboard['artist.inverted'].unique()

In [ ]:
# Easier to visualize!!
b_df[(b_df['artist.inverted'] == 'Jay-Z')]

In [ ]:
b_df[(b_df['artist.inverted'] == 'Jay-Z')]['rank'].plot.hist(bins=100, title="Jay-Z Hits by Week")

In [ ]:
#Scatterplot of JayZ at different songs/days?
b_df[(b_df['artist.inverted'] == 'Jay-Z')].plot.scatter(y='rank', 
                                                        x='date',
                                                        rot=45)

In [ ]:
import seaborn as sns
g = sns.pairplot(x_vars=["date"], 
             y_vars=["rank"], 
             data=b_df[(b_df['artist.inverted'] == 'TLC')], 
             hue="track",
             height=5)

# # Hacky rotation?? -- https://github.com/mwaskom/seaborn/issues/867
for ax in g.axes.flat:
    for label in ax.get_xticklabels():
        label.set_rotation(45)

# # Invert..
for ax in g.axes.flat:
    ax.invert_yaxis()

In [ ]:
# Or we can get the list of top ranked songs...
b_df[(b_df['rank'] == 1.0)].groupby(['artist.inverted']).count()